In [ ]:
from tensorflow.keras import layers, Input, Model
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from models.DIL_unet1D_regressor import encoding_block, decoding_block

In [ ]:
N = 144
input_features = 1
kernel_size = 3
tau = 1
dilation_rate = 1

predicted_points = 1


In [ ]:
import numpy as np
x =np.array([48, 264, 269, 270, 271, 268, 269, 269, 267, 266, 263,
       265, 265, 261, 256, 248, 242, 236, 229, 221, 214, 207,
       199, 192, 185, 179, 172, 166, 160, 155, 150, 145, 140,
       136, 127, 120, 114, 115, 114, 113, 112, 112, 109, 108,
       108, 108, 110, 103, 105, 108, 111, 114, 118, 123, 126,
       128, 124, 132, 131, 129, 127, 125, 115, 104,  96,  89,
        85,  81,  77,  66,  61,  59,  61,  68,  64,  58,  55,
        55,  58,  61,  69,  73,  73,  79,  95, 114, 132, 143,
       156, 174, 186, 193, 196, 197, 195, 189, 185, 180, 170,
       162, 153, 148, 141, 135, 128, 123, 119, 117, 115, 114,
       111, 109, 106, 104, 103, 100,  99,  92,  91,  86,  80,
        72,  63,  60,  65,  69,  79,  87,  94, 103, 105, 103,
       106, 103,  99,  93,  89,  87,  87,  86,  89,  93,  99,
       103])
# to float 
x = x.astype(np.float32)

In [55]:
# Returns a CNN-model instance 
def get_model(N: int = 144, input_features: int = 1,
              tau : int = 1, kernel_size : int = 3, dilation_rate : int = 1, predicted_points : int = 1) -> Model:
    """Returns the model described in [1].

    Args:
    -----
        N (int): Number of samples in the input tensor. Must be multiple of 2. Default: CGM_INPUT_POINTS.
        input_features (int): Number of features in the input tensor. Default: NUMBER_OF_INPUT_SIGNALS.
        tau (int): Stride of the convolutional layers. Default: 1, as [1]
        kernel_size (int): Kernel size of the convolutional layers. Default: 3, as [1]
        output_points (int): Number of predictied points (time dimension) Default: 1.
    
    Returns:
    --------
        model (Model): CNN-model instance.
    
    References:
    -----------
        [1] F. Renna, J. Oliveira and M. T. Coimbra, "Deep Convolutional Neural
        Networks for Heart Sound Segmentation," in IEEE Journal of Biomedical
        and Health Informatics, vol. 23, no. 6, pp. 2435-2445, Nov. 2019, doi:
        10.1109/JBHI.2019.2894222.
    """
    # Input tensor
    input = Input(shape=(N, input_features)) 

    # Encoding phase of the network: downsampling inputs 
    x, res_1 = encoding_block(input, filters=input_features*2, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "enc_0")
    x, res_2 = encoding_block(x, filters=input_features*4, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "enc_1")
    x, res_3 = encoding_block(x, filters=input_features*8, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "enc_2")
    x, res_4 = encoding_block(x, filters=input_features*16, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "enc_3")

    # Intermediate layer 
    # 2 x ( Conv + ReLU )
    x = layers.Conv1D(filters=input_features*32, kernel_size=kernel_size, strides=1, activation="relu", padding="same", dilation_rate = dilation_rate, name='central_conv_relu_0')(x)
    x = layers.Conv1D(filters=input_features*32, kernel_size=kernel_size, strides=1, activation="relu", padding="same", dilation_rate = dilation_rate, name='central_conv_relu_1')(x)
       
    # Decoding phase of the network: upsampling inputs
    x = decoding_block(x, res_4, filters=input_features*16, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "dec_0")
    x = decoding_block(x, res_3, filters=input_features*8, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "dec_1")
    x = decoding_block(x, res_2, filters=input_features*4, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "dec_2")
    x = decoding_block(x, res_1, filters=input_features*2, kernel_size=kernel_size, stride = tau, activation = "relu", padding = "same", dilation_rate = dilation_rate, name_prefix = "dec_3")

    # Output of the model (modified from [1] to switch from classification to regression) 
    x = layers.Conv1D(filters=predicted_points, kernel_size=kernel_size, strides=tau, padding="same", dilation_rate = dilation_rate, name='final_conv')(x)

    # Ouput layer is a regression layer of 1 point
    # output = layers.Dense(predicted_points)(x)

    # Reshape x to be a 3D tensor
    x = layers.Reshape((1, N), input_shape=(N, 1))(x)

    # Add timeDistributed layer
    x = layers.TimeDistributed(layers.Dense(N))(x)
    x = layers.TimeDistributed(layers.Dense(32))(x)
    x = layers.TimeDistributed(layers.Dense(8))(x)
    x = layers.TimeDistributed(layers.Dense(2))(x)

    # Once flattened, add a dense layer to predict the output
    output = layers.Dense(predicted_points)(x)

    # Define the model
    model = Model(input, output)

X before reshape:  (None, 144, 1)
X after reshape:  (None, 1, 144)
X after timeDistributed 1:  (None, 1, 144)
X after timeDistributed 2:  (None, 1, 32)
X after timeDistributed 3:  (None, 1, 8)
X after timeDistributed 4:  (None, 1, 2)
Output shape:  (None, 1, 1)


In [ ]:
get_model